In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import sqlite3
from sqlalchemy import create_engine
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing

con = sqlite3.connect("/content/gdrive/My Drive/Colab Notebooks/wildfires.sqlite")

In [12]:
CATEGORICAL = ["NWCG_REPORTING_AGENCY", "FIRE_SIZE_CLASS", "STATE"]
NUMERICAL = ["FIRE_YEAR", "DISCOVERY_DOY", "DISCOVERY_DATE", "STAT_CAUSE_CODE", "CONT_DOY", "CONT_DATE", "FIRE_SIZE", "LATITUDE", "LONGITUDE", "OWNER_CODE"]
fires_table = pd.read_sql_query("SELECT * from Fires", con)
wildfires = fires_table[NUMERICAL + CATEGORICAL]

In [13]:
wildfires.size

24446045

In [40]:
large_wildfires = wildfires.loc[wildfires["FIRE_SIZE"] > 10]
large_wildfires.size

3106415

In [46]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
d = defaultdict(LabelEncoder)

# Encoding the variable
large_wildfires = large_wildfires.apply(lambda x: d[x.name].fit_transform(x))

# # Inverse the encoded
# fit.apply(lambda x: d[x.name].inverse_transform(x))

# # Using the dictionary to label future data
# df.apply(lambda x: d[x.name].transform(x))

In [76]:
from sklearn.model_selection import train_test_split

x = large_wildfires.drop(["FIRE_SIZE_CLASS", "FIRE_SIZE"], 1)
X_train, X_test, y_train, y_test = train_test_split(x, large_wildfires["FIRE_SIZE_CLASS"], test_size=0.2, random_state=15)

In [77]:
X_train.size, y_train.size

(2102804, 191164)

As a baseline classifier, we simply assign the most common fire size class to each fire. Our models must improve upon this number.

In [78]:
large_wildfires["FIRE_SIZE_CLASS"].value_counts().max() / large_wildfires["FIRE_SIZE_CLASS"].size

0.7736268335042162

In [79]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 47791 points : 14699


In [80]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [81]:
# tree.plot_tree(clf) 

[Text(163.73788429088503, 215.59728813559323, 'X[7] <= 70270.5\ngini = 0.382\nsamples = 191164\nvalue = [147946, 22690, 11304, 6227, 2997]'),
 Text(89.35769406851401, 211.91186440677967, 'X[3] <= 0.5\ngini = 0.591\nsamples = 67347\nvalue = [40412, 11552, 7477, 5112, 2794]'),
 Text(43.49066658624935, 208.2264406779661, 'X[7] <= 28004.5\ngini = 0.699\nsamples = 20808\nvalue = [9977, 3554, 2952, 2490, 1835]'),
 Text(30.418982222987058, 204.54101694915255, 'X[6] <= 166553.5\ngini = 0.741\nsamples = 11480\nvalue = [4785, 1847, 1705, 1664, 1479]'),
 Text(21.996943994324592, 200.855593220339, 'X[4] <= 243.5\ngini = 0.726\nsamples = 9727\nvalue = [4279, 1628, 1436, 1373, 1011]'),
 Text(12.17117317816333, 197.17016949152543, 'X[9] <= 3.5\ngini = 0.699\nsamples = 6661\nvalue = [3194, 1123, 929, 844, 571]'),
 Text(2.9050389648845325, 193.48474576271187, 'X[9] <= 0.5\ngini = 0.737\nsamples = 4260\nvalue = [1787, 748, 641, 640, 444]'),
 Text(0.450838449862143, 189.7993220338983, 'X[7] <= 6720.0\ngi

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fd8aa8f2510> (for post_execute):


KeyboardInterrupt: ignored

Error in callback <function flush_figures at 0x7fd8a97e3a60> (for post_execute):


KeyboardInterrupt: ignored

In [83]:
pred = clf.predict(X_test)

In [84]:
np.mean(pred != y_test)

0.3203950534619489